In [6]:
from nltk.corpus import udhr
from nltk.util import ngrams
import nltk
import string
import re


def data_preprocess(text):
    text = str.lower(text.translate(str.maketrans('','',string.punctuation)).strip())
    text = re.sub('\n',' ', text) 
    return text

#calulate probabilty distribution from frequency distribution
def find_probablity(freq_dist):
    total_count = 0
    for key in freq_dist:
        total_count = total_count + freq_dist[key]
    probablity_dict = {}
    for key in freq_dist:
        probablity_dict[key] = freq_dist[key]/total_count
    return probablity_dict

#Function to calculate the probablity of Unigram model
def Find_Unigram_Prob(data, Lang):
    count = 1
    uni_data = Uni_Probability[Lang]
    for ch in data:
        key = (ch,)
        count_no_of_key = uni_data[key] if key in uni_data else 0
        
        if count_no_of_key == 0:
            return 0
        
        count  = count * count_no_of_key
        
    return count

#Function to calculate the probablity of Bi model
def Find_Bigram_Prob(data, Lang):
    data_len = len(data)    
    if data_len < 2:
        return Find_Unigram_Prob(data, Lang)
    
    count = 1
    bi_data = Bi_Probability[Lang]
    
    for i in range(data_len+1):
        if i == 0:
            key = (' ',data[i])
        elif i == data_len:
            key == (data[i-1], ' ')
        else:
            key = (data[i-1],data[i])
        
        count_no_of_key = bi_data[key] if key in bi_data else 0
        
        if count_no_of_key == 0:
            return 0
        
        prev_char = ' ' if i == 0 else data[i-1]
        
        count = count * count_no_of_key / Find_Unigram_Prob(str(prev_char), Lang)
    
    return count;

#Function to calculate the probablity of trigram model
def Find_Trigram_Prob(data, Lang):
    data_len = len(data)
    if data_len < 3:
        return Find_Bigram_Prob(data, Lang)
    
    count = 1
    tri_data = Tri_Probability[Lang]
    
    for i in range(1, data_len+1):
        if i == 1:
            key = (' ', data[i-1], data[i])
        elif i == data_len:
            key = (data[i-2], data[i-1], ' ')
        else:
            key = (data[i-2], data[i-1], data[i])
        
        count_no_of_key = tri_data[key] if key in tri_data else 0
        
        if count_no_of_key == 0:
            return 0
        
        new_key = (' ', data[i-1]) if i == 1 else (data[i-2], data[i-1])
        count_no_of_newkey = Bi_Probability[Lang][new_key]
        
        count = count * count_no_of_key / count_no_of_newkey
    
    return count

#Function to calculate ngram model accuracies of the given data_set
def Language_Pred_Model(data_set, First_Lang, Sec_Lang):
    count_unigrams = 0
    count_bigrams = 0
    count_trigrams = 0
    
    for word in data_set:
        uni_prob_lang1 = Find_Unigram_Prob(word, First_Lang)
        uni_prob_lang2 = Find_Unigram_Prob(word, Sec_Lang)
        
        bi_prob_lang1 = Find_Bigram_Prob(word, First_Lang)
        bi_prob_lang2 = Find_Bigram_Prob(word, Sec_Lang)
        
        tri_prob_lang1 = Find_Trigram_Prob(word, First_Lang)
        tri_prob_lang2 = Find_Trigram_Prob(word, Sec_Lang)
    
        
        if uni_prob_lang1 >= uni_prob_lang2:
            count_unigrams = count_unigrams + 1
        
        if bi_prob_lang1 >= bi_prob_lang2:
            count_bigrams = count_bigrams + 1
        
        
        if tri_prob_lang1 >= tri_prob_lang2:
            count_trigrams = count_trigrams + 1
    
    print("-----------------------------------------------------------------------")
    print("Accuracy for Unigram Model:", ((count_unigrams * 100) / len(data_set)))
    print("Accuracy for Bigram Model:", ((count_bigrams * 100) / len(data_set)))
    print("Accuracy for Trigram Model:", ((count_trigrams * 100) / len(data_set)))


english = udhr.raw('English-Latin1') 
french = udhr.raw('French_Francais-Latin1') 
italian = udhr.raw('Italian_Italiano-Latin1') 
spanish = udhr.raw('Spanish_Espanol-Latin1')  

corpus_data_all = [data_preprocess(lang) for lang in [english, french, italian, spanish]]


english = corpus_data_all[0]
french = corpus_data_all[1]
italian = corpus_data_all[2]
spanish = corpus_data_all[3]


english_train, english_dev = english[0:1000], english[1000:1100]
french_train, french_dev = french[0:1000], french[1000:1100]
italian_train, italian_dev = italian[0:1000], italian[1000:1100]
spanish_train, spanish_dev = spanish[0:1000], spanish[1000:1100]  
english_test = udhr.words('English-Latin1')[0:1000]
french_test = udhr.words('French_Francais-Latin1')[0:1000]
italian_test = udhr.words('Italian_Italiano-Latin1')[0:1000]
spanish_test = udhr.words('Spanish_Espanol-Latin1')[0:1000]

#train data for all languages
train_data = [english_train,french_train,
         italian_train,spanish_train]

#development data for all lenguages
dev_data = [english_dev,french_dev,
       italian_dev,spanish_dev]

#test data for all lenguages
test_data = [english_test,french_test,
        italian_test, spanish_test]

#Test data being taken as raw data it needs to be preprocessed before using
test_data = [[data_preprocess(word) for word in test_sample] for test_sample in test_data]


Unigrams_Train_Data = [ngrams(train_data[language], 1) 
            for language in Languages_List]

Bigrams_Train_Data = [ngrams(train_data[language], 2, 
                  pad_left=True, pad_right=True, 
                  left_pad_symbol=' ', right_pad_symbol=' ') 
            for language in Languages_List]

Trigrams_Train_Data = [ngrams(train_data[language], 3, 
                  pad_left=True, pad_right=True, 
                  left_pad_symbol=' ', right_pad_symbol=' ') 
            for language in Languages_List]


#Finding Frequency Distribution and Probablities using the calculated Frequency Distribution of Unigrams, Bigrams , Trigrams Calculated 
Unigram_Freq = [nltk.FreqDist(unigram_data) for unigram_data in Unigrams_Train_Data]
Bigram_Freq = [nltk.FreqDist(bigram_data) for bigram_data in Bigrams_Train_Data]
Trigram_Freq = [nltk.FreqDist(trigram_data) for trigram_data in Trigrams_Train_Data]
Uni_Probability = [find_probablity(freqdist) for freqdist in Unigram_Freq]
Bi_Probability = [find_probablity(freqdist) for freqdist in Bigram_Freq]
Tri_Probability = [find_probablity(freqdist) for freqdist in Trigram_Freq]


print("DEV RESULTS:")
print("\n")
print("*English-French : ")
Language_Pred_Model(dev_data[0].split(), ENGLISH, FRENCH);
print("\n")
print("*Spanish-Italian : ")
Language_Pred_Model(dev_data[1].split(), SPANISH, ITALIAN);


print("\n")
print("************************************************************************")
print("\n")
print("TEST OUTPUT:")
print("\n")
print("*English-French : ")
#Question 1 - Selected and tested model for English and French Languages
Language_Pred_Model(test_data[0], ENGLISH, FRENCH);
print("\n")
print("*Spanish-Italian : ")
#Question 2 - Selected and tested model for Spanish and Italian languages
Language_Pred_Model(test_data[3], SPANISH, ITALIAN);

DEV RESULTS:


*English-French : 
-----------------------------------------------------------------------
Accuracy for Unigram Model: 70.58823529411765
Accuracy for Bigram Model: 94.11764705882354
Accuracy for Trigram Model: 100.0


*Spanish-Italian : 
-----------------------------------------------------------------------
Accuracy for Unigram Model: 64.70588235294117
Accuracy for Bigram Model: 76.47058823529412
Accuracy for Trigram Model: 94.11764705882354


************************************************************************


TEST OUTPUT:


*English-French : 
-----------------------------------------------------------------------
Accuracy for Unigram Model: 78.2
Accuracy for Bigram Model: 92.1
Accuracy for Trigram Model: 99.0


*Spanish-Italian : 
-----------------------------------------------------------------------
Accuracy for Unigram Model: 67.0
Accuracy for Bigram Model: 84.6
Accuracy for Trigram Model: 96.3


#Question:
Which language pair is harder to distinguish?
Calculating the accuracies of the Unigram, Bigram and Trigram models for all the language models it is obseved that the accuracy for the language pair of Spanish - Italian is harder.